# Remote Execution with ras-commander

This notebook demonstrates how to execute HEC-RAS plans using:
1. **Local parallel execution** - `RasCmdr.compute_parallel()` on your local machine
2. **Remote execution** - `compute_parallel_remote()` on remote machines via PsExec/Docker

## Features
- Distributed execution across multiple remote machines
- Automatic project deployment via network shares
- Parallel execution with configurable workers
- Result collection and consolidation
- **Automatic PsExec.exe download** (no manual setup required)

## Requirements for Remote Execution
- Remote machine(s) configured per `REMOTE_WORKER_SETUP_GUIDE.md`
- Network share accessible from control machine
- HEC-RAS installed on remote machine(s)

### Critical Configuration Requirements

**IMPORTANT**: Remote HEC-RAS execution requires session-based execution. Never use `system_account=True`.

**Required on Remote Machine**:
1. **Session ID Configuration**: Typically `session_id=2` for interactive sessions
   - Query session: `query session /server:HOSTNAME` to find correct ID
2. **Group Policy Rights** (on remote machine):
   - Access this computer from the network
   - Allow log on locally
   - Log on as a batch job
3. **Registry Setting**: `LocalAccountTokenFilterPolicy=1`
   ```powershell
   New-ItemProperty -Path "HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System" `
       -Name "LocalAccountTokenFilterPolicy" -Value 1 -PropertyType DWORD -Force
   ```
4. **Remote Registry Service**: Must be running
5. **User Permissions**: User must be in Administrators group

**Why This Matters**: HEC-RAS is a GUI application and will fail silently if executed in system account context.

### Reference Documentation
- **ras-commander Remote Execution**: `.claude/rules/hec-ras/remote.md` (critical configuration patterns)
- **Worker Setup Guide**: `feature_dev_notes/RasRemote/REMOTE_WORKER_SETUP_GUIDE.md`
- **HEC-RAS System Requirements**: [HEC-RAS Downloads](https://www.hec.usace.army.mil/software/hec-ras/download.aspx)

### Security Considerations
- **Network Share Access**: Ensure proper permissions and consider encryption for sensitive models
- **Credential Management**: Use secure credential storage (not hardcoded passwords)
- **Firewall Rules**: Remote execution requires SMB (port 445) and RPC access
- **Audit Trail**: All remote executions are logged for review

---
# Part 1: Setup and Imports

In [1]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
from pathlib import Path  # Import Path unconditionally
import os
import time

USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander core functionality
from ras_commander import HdfResultsPlan, RasCmdr, RasExamples, init_ras_project, ras

# Import remote execution functions (lazy loaded)
from ras_commander import load_workers_from_json, compute_parallel_remote

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

📦 PIP PACKAGE MODE: Loading installed ras-commander
✓ Loaded: c:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\ras_commander\__init__.py


## Parameters

Configure these values to customize the notebook for your project.

In [2]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "BaldEagleCrkMulti2D"           # Example project to extract
RAS_VERSION = "6.6"               # HEC-RAS version (6.3, 6.5, 6.6, etc.)

# Remote Execution Settings
REMOTE_HOST = "192.168.1.100"     # Remote machine hostname/IP
REMOTE_USER = "ras_user"          # Username on remote machine
SESSION_ID = 2                    # Windows session ID (typically 2)
REMOTE_SHARE = r"\\{host}\RAS_Share"  # UNC path to shared folder

print(f"Configuration: {PROJECT_NAME} project, RAS {RAS_VERSION}")

Configuration: BaldEagleCrkMulti2D project, RAS 6.6


In [3]:
# Extract example project for testing
project_path = RasExamples.extract_project(PROJECT_NAME, suffix="500")
print(f"Project extracted to: {project_path}")

# Initialize project
ras = init_ras_project(project_path, RAS_VERSION)
print(f"\nProject: {ras.project_name}")
print(f"Available plans: {list(ras.plan_df['plan_number'])}")

2025-12-29 06:06:33 - ras_commander.RasExamples - INFO - Found zip file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\Example_Projects_6_6.zip
2025-12-29 06:06:33 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-12-29 06:06:33 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-12-29 06:06:33 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-12-29 06:06:33 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D' as 'BaldEagleCrkMulti2D_500'
2025-12-29 06:06:33 - ras_commander.RasExamples - INFO - Folder 'BaldEagleCrkMulti2D_500' already exists. Deleting existing folder...
2025-12-29 06:06:33 - ras_commander.RasExamples - INFO - Existing folder 'BaldEagleCrkMulti2D_500' has been deleted.
2025-12-29 06:06:36 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site

Project extracted to: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500

Project: BaldEagleDamBrk
Available plans: ['13', '15', '17', '18', '19', '03', '04', '02', '01', '05', '06']


---
# Part 2: Local Parallel Execution

Use `RasCmdr.compute_parallel()` to run multiple plans on your **local machine**.
This is the simplest way to parallelize HEC-RAS execution.

In [4]:
LOCAL_PLANS = ["03", "04", "06"]  # Plans to execute
LOCAL_MAX_WORKERS = 3              # Number of parallel processes
LOCAL_NUM_CORES = 2                # Cores per HEC-RAS instance

print(f"Plans to execute: {LOCAL_PLANS}")
print(f"Configuration: {LOCAL_NUM_CORES} cores x {LOCAL_MAX_WORKERS} workers")

Plans to execute: ['03', '04', '06']
Configuration: 2 cores x 3 workers


In [5]:
print(f"Starting LOCAL parallel execution of {len(LOCAL_PLANS)} plans...")
print("="*70)

start_time = time.time()

local_results = RasCmdr.compute_parallel(
    plan_number=LOCAL_PLANS,
    max_workers=LOCAL_MAX_WORKERS,
    num_cores=LOCAL_NUM_CORES,
    ras_object=ras,
    overwrite_dest=True
)

elapsed = time.time() - start_time

print(f"\nExecution complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")

2025-12-29 06:06:36 - ras_commander.RasCmdr - INFO - Filtered plans to execute: ['03', '04', '06']
2025-12-29 06:06:36 - ras_commander.RasCmdr - INFO - Adjusted max_workers to 3 based on the number of plans to compute: 3


Starting LOCAL parallel execution of 3 plans...


2025-12-29 06:06:36 - ras_commander.RasCmdr - INFO - Created worker folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500 [Worker 1]
2025-12-29 06:06:36 - ras_commander.rasmap - INFO - Successfully parsed RASMapper file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500 [Worker 1]\BaldEagleDamBrk.rasmap
2025-12-29 06:06:36 - ras_commander.RasCmdr - INFO - Created worker folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500 [Worker 2]
2025-12-29 06:06:36 - ras_commander.rasmap - INFO - Successfully parsed RASMapper file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500 [Worker 2]\BaldEagleDamBrk.rasmap
2025-12-29 06:06:37 - ras_commander.RasCmdr - INFO - Created worker folder: C:\Users\billk_clb\anaconda3\envs\


Execution complete in 896.1 seconds (14.9 minutes)


In [6]:
print("LOCAL Execution Results:")
print("="*70)

success_count = sum(1 for success in local_results.values() if success)
fail_count = len(local_results) - success_count

for plan_num, success in local_results.items():
    status = "SUCCESS" if success else "FAILED"
    print(f"  Plan {plan_num}: {status}")

print(f"\nSummary: {success_count}/{len(local_results)} plans succeeded")
print(f"Results folder: {ras.project_folder.parent / (ras.project_folder.name + ' [Computed]')}")

LOCAL Execution Results:
  Plan 03: SUCCESS
  Plan 04: SUCCESS
  Plan 06: SUCCESS

Summary: 3/3 plans succeeded
Results folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500 [Computed]


---
# Part 3: Remote Execution Setup

To use **remote execution**, you need to configure remote workers.

## First Time Setup
1. Copy `RemoteWorkers.json.template` to `RemoteWorkers.json`
2. Edit `RemoteWorkers.json` with your remote machine details
3. The JSON file is in `.gitignore` for security

## Worker Types
- **local** - Execute on local machine (useful for mixed local+remote)
- **psexec** - Execute on remote Windows via PsExec
- **docker** - Execute in Docker container (local or remote via SSH)

## JSON Configuration Example
```json
{
  "workers": [
    {
      "name": "Local Worker",
      "worker_type": "local",
      "worker_folder": "C:\\RasRemote",
      "cores_total": 8,
      "cores_per_plan": 2,
      "enabled": true
    },
    {
      "name": "Remote Workstation",
      "worker_type": "psexec",
      "hostname": "192.168.1.100",
      "share_path": "\\\\192.168.1.100\\RasRemote",
      "worker_folder": "C:\\RasRemote",
      "username": "your_username",
      "password": "your_password",
      "session_id": 2,
      "cores_total": 16,
      "cores_per_plan": 4,
      "enabled": true
    }
  ]
}
```

In [7]:
# Check if RemoteWorkers.json exists
config_file = Path("RemoteWorkers.json")

if not config_file.exists():
    print("WARNING: RemoteWorkers.json not found!")
    print()
    print("To use remote execution:")
    print("1. Copy RemoteWorkers.json.template to RemoteWorkers.json")
    print("2. Edit RemoteWorkers.json with your remote machine details")
    print("3. Re-run this cell")
    print()
    print("For now, you can still use LOCAL parallel execution (Part 2 above).")
    REMOTE_AVAILABLE = False
else:
    import json
    with open(config_file, 'r') as f:
        worker_configs = json.load(f)
    
    enabled_configs = [w for w in worker_configs["workers"] if w.get("enabled", True)]
    
    print(f"Found {len(enabled_configs)} enabled worker(s) in RemoteWorkers.json:")
    for w in enabled_configs:
        cores_total = w.get('cores_total', 'Not set')
        cores_per_plan = w.get('cores_per_plan', 4)
        print(f"  - {w.get('name', 'unnamed')} ({w.get('worker_type', 'unknown')})")
        print(f"    Cores: {cores_total} total, {cores_per_plan} per plan")
    
    REMOTE_AVAILABLE = True

Found 3 enabled worker(s) in RemoteWorkers.json:
  - CLB-04 (psexec)
    Cores: 8 total, 2 per plan
  - Local Compute (local)
    Cores: 8 total, 2 per plan
  - CLB-04 Docker 6.6 (docker)
    Cores: 4 total, 4 per plan


---
# Part 4: Remote Parallel Execution

Use `compute_parallel_remote()` to run plans on **remote machines**.

**Key differences from local execution:**
- Plans are copied to remote shares
- Execution happens on remote machines
- Results are collected back to local project
- Supports multiple remote workers simultaneously

In [8]:
# Load remote workers from JSON
# NOTE: This must be called AFTER init_ras_project() so ras_exe_path is available

if REMOTE_AVAILABLE:
    workers = load_workers_from_json("RemoteWorkers.json")
    
    print(f"Loaded {len(workers)} worker(s):")
    for w in workers:
        print(f"  - {w.worker_id} ({w.worker_type})")
        if hasattr(w, 'max_parallel_plans') and w.max_parallel_plans > 1:
            print(f"    Parallel Capacity: {w.max_parallel_plans} plans")
else:
    print("Remote workers not available - configure RemoteWorkers.json first")
    workers = []

2025-12-29 06:21:32 - ras_commander.remote.RasWorker - INFO - Initializing psexec worker
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO - Initializing PsExec worker for 192.168.3.8
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO - PsExec worker configured:
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   Hostname: 192.168.3.8
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   Share path: \\192.168.3.8\RasRemote
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   Worker folder: C:\RasRemote
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   User: .\bill
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   System account: False
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   Session ID: 2
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   Process Priority: low
2025-12-29 06:21:32 - ras_commander.remote.PsexecWorker - INFO -   Queue Priority: 1
2

Loaded 2 worker(s):
  - CLB-04 (psexec)
    Parallel Capacity: 4 plans
  - Local Compute (local)
    Parallel Capacity: 4 plans


In [9]:
# Configuration for remote execution
REMOTE_PLANS = ["03", "04", "06"]  # Plans to execute remotely
REMOTE_NUM_CORES = 4               # Cores per HEC-RAS instance

print(f"Plans to execute: {REMOTE_PLANS}")
print(f"Workers available: {len(workers)}")

Plans to execute: ['03', '04', '06']
Workers available: 2


In [10]:
# Execute plans on REMOTE workers
if workers:
    print(f"Starting REMOTE execution of {len(REMOTE_PLANS)} plans...")
    print(f"Using {len(workers)} worker(s)")
    print("="*70)
    
    start_time = time.time()
    
    remote_results = compute_parallel_remote(
        plan_numbers=REMOTE_PLANS,
        workers=workers,
        num_cores=REMOTE_NUM_CORES,
        autoclean=True  # Delete temp folders after execution (default)
    )
    
    elapsed = time.time() - start_time
    
    print(f"\nExecution complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
else:
    print("No remote workers available.")
    print("Configure RemoteWorkers.json to enable remote execution.")
    remote_results = {}

2025-12-29 06:21:32 - ras_commander.remote.Execution - INFO - Starting distributed execution of 3 plans across 2 workers


Starting REMOTE execution of 3 plans...
Using 2 worker(s)


2025-12-29 06:21:32 - ras_commander.remote.Execution - INFO - Total worker slots available: 8
2025-12-29 06:21:32 - ras_commander.remote.Execution - INFO - Submitting plan 03 to worker Local Compute (sub-worker #1)
2025-12-29 06:21:32 - ras_commander.remote.LocalWorker - INFO - Starting local execution of plan 03 (sub-worker #1)
2025-12-29 06:21:32 - ras_commander.remote.Execution - INFO - Submitting plan 04 to worker Local Compute (sub-worker #2)
2025-12-29 06:21:32 - ras_commander.remote.LocalWorker - INFO - Copying project to C:\RasRemote\BaldEagleDamBrk_03_SW1_140a1b33
2025-12-29 06:21:32 - ras_commander.remote.LocalWorker - INFO - Starting local execution of plan 04 (sub-worker #2)
2025-12-29 06:21:32 - ras_commander.remote.Execution - INFO - Submitting plan 06 to worker Local Compute (sub-worker #3)
2025-12-29 06:21:32 - ras_commander.remote.LocalWorker - INFO - Copying project to C:\RasRemote\BaldEagleDamBrk_04_SW2_4b9689d9
2025-12-29 06:21:32 - ras_commander.remote.LocalWorker 


Execution complete in 880.6 seconds (14.7 minutes)


In [11]:
# Display remote execution results
if remote_results:
    print("REMOTE Execution Results:")
    print("="*70)
    
    for plan_num, result in remote_results.items():
        if result.success:
            print(f"  Plan {plan_num}: SUCCESS ({result.execution_time:.1f}s)")
            print(f"    HDF Path: {result.hdf_path}")
        else:
            print(f"  Plan {plan_num}: FAILED - {result.error_message}")
    
    success_count = sum(1 for r in remote_results.values() if r.success)
    print(f"\nSummary: {success_count}/{len(remote_results)} plans succeeded")
else:
    print("No remote results - run the remote execution cell above first.")

REMOTE Execution Results:
  Plan 03: SUCCESS (740.1s)
    HDF Path: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500\BaldEagleDamBrk.p03.hdf
  Plan 06: SUCCESS (847.6s)
    HDF Path: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500\BaldEagleDamBrk.p06.hdf
  Plan 04: SUCCESS (880.5s)
    HDF Path: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500\BaldEagleDamBrk.p04.hdf

Summary: 3/3 plans succeeded


## Verification and Validation

### What to Verify After Remote Execution

**1. Remote Connection Success**
- ✓ Worker connected successfully (check session ID in logs)
- ✓ Project files deployed to remote share
- ✓ HEC-RAS process started on remote machine

**2. Execution Status**
- ✓ Return code = 0 (success)
- ✓ HDF file created on remote machine
- ✓ Compute messages indicate "Run completed successfully"
- ✓ No errors in execution log

**3. File Transfer Validation**
- ✓ HDF results copied back to local machine
- ✓ File size matches expected (not truncated)
- ✓ HDF file is readable and contains results

**4. Result Quality**
- ✓ Check compute messages for warnings
- ✓ Verify WSE/depth values are reasonable
- ✓ No NaN values in critical areas
- ✓ Hydraulic stability (no excessive oscillations)

### Audit Trail
Save these artifacts for professional review:
- **Execution logs**: `logs/remote_execution_{timestamp}.log`
- **Compute messages**: Extract from HDF and save as text
- **Return codes**: Document success/failure for each worker
- **Network transfer logs**: Verify data integrity

### LLM Forward Principle
Remote execution creates **multiple review pathways**:
1. **Traditional Engineering Review**: Open HDF in HEC-RAS GUI
2. **Visual Inspection**: Plot results (WSE, velocity, etc.)
3. **Code Audit Trail**: Review execution logs and parameters

---
# Part 5: Verify Results

Check that HDF files were created and contain valid results.

In [12]:
# Verify HDF results
from ras_commander import HdfResultsPlan

print("Result Verification:")
print("="*70)

# Check plans that were executed (either local or remote)
plans_to_check = LOCAL_PLANS if local_results else (REMOTE_PLANS if remote_results else [])

for plan_num in plans_to_check:
    hdf_path = project_path / f"{ras.project_name}.p{plan_num}.hdf"
    
    if hdf_path.exists():
        size_mb = hdf_path.stat().st_size / (1024 * 1024)
        print(f"\nPlan {plan_num}:")
        print(f"  HDF Size: {size_mb:.2f} MB")
        
        # Get compute messages
        msgs = HdfResultsPlan.get_compute_messages(hdf_path)
        if "completed successfully" in msgs.lower() or "complete process" in msgs.lower():
            print(f"  Status: SUCCESS")
        else:
            print(f"  Status: Check messages")
        
        # Get volume accounting
        vol = HdfResultsPlan.get_volume_accounting(hdf_path)
        if vol is not None and len(vol) > 0:
            error_pct = vol['Error Percent'].iloc[0]
            print(f"  Volume Error: {error_pct:.4f}%")
    else:
        print(f"\nPlan {plan_num}: HDF not found")

2025-12-29 06:36:13 - ras_commander.hdf.HdfResultsPlan - INFO - Using existing Path object HDF file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500\BaldEagleDamBrk.p03.hdf
2025-12-29 06:36:13 - ras_commander.hdf.HdfResultsPlan - INFO - Final validated file path: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500\BaldEagleDamBrk.p03.hdf
2025-12-29 06:36:13 - ras_commander.hdf.HdfResultsPlan - INFO - Reading computation messages from HDF: BaldEagleDamBrk.p03.hdf
2025-12-29 06:36:13 - ras_commander.hdf.HdfResultsPlan - INFO - Successfully extracted 2013 characters from HDF
2025-12-29 06:36:13 - ras_commander.hdf.HdfResultsPlan - INFO - Using existing Path object HDF file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_500\BaldEagleDamBrk.p03.hdf
2025-12-29 06:36:13 - ras_commander.hdf.Hd

Result Verification:

Plan 03:
  HDF Size: 59.07 MB
  Status: SUCCESS
  Volume Error: 0.0187%

Plan 04:
  HDF Size: 81.73 MB
  Status: SUCCESS
  Volume Error: 0.0007%

Plan 06:
  HDF Size: 577.57 MB
  Status: SUCCESS
  Volume Error: 0.0003%


---
# Part 6: Manual Worker Configuration (Optional)

You can also create workers programmatically without a JSON file.

In [13]:
# Example: Create a PsExec worker manually
# Uncomment and modify with your settings to test

# manual_worker = init_ras_worker(
#     "psexec",
#     hostname="192.168.1.100",
#     share_path=r"\\192.168.1.100\RasRemote",
#     worker_folder=r"C:\RasRemote",
#     credentials={
#         "username": "your_username",
#         "password": "your_password"
#     },
#     session_id=2,
#     cores_total=8,
#     cores_per_plan=2
# )
# 
# print(f"Manual worker created: {manual_worker.worker_id}")
# print(f"  Parallel Capacity: {manual_worker.max_parallel_plans} plans")

print("Uncomment the code above to create a manual worker.")

Uncomment the code above to create a manual worker.


---
# Part 7: Cleanup (Optional)

Clean up temporary worker folders on remote shares.

In [14]:
# Cleanup function for remote shares
def cleanup_remote_shares(workers, dry_run=True):
    """Clean up worker folders from remote shares."""
    import shutil
    
    seen_shares = set()
    
    for w in workers:
        if not hasattr(w, 'share_path') or not w.share_path:
            continue
            
        share_path = Path(w.share_path)
        if str(share_path) in seen_shares:
            continue
        seen_shares.add(str(share_path))
        
        # Check if share exists before accessing
        if not share_path.exists():
            continue
        
        try:
            folders = [f for f in share_path.iterdir() if f.is_dir()]
            
            print(f"Share: {share_path}")
            print(f"  Folders: {len(folders)}")
            
            for folder in folders:
                folder_size = sum(f.stat().st_size for f in folder.rglob('*') if f.is_file())
                folder_size_mb = folder_size / (1024 * 1024)
                
                if dry_run:
                    print(f"  [WOULD DELETE] {folder.name} ({folder_size_mb:.1f} MB)")
                else:
                    print(f"  [DELETING] {folder.name}")
                    shutil.rmtree(folder, ignore_errors=True)
                    
        except Exception as e:
            print(f"Error accessing {share_path}: {e}")

# Preview cleanup (dry run)
if workers:
    print("CLEANUP PREVIEW (dry_run=True):")
    cleanup_remote_shares(workers, dry_run=True)
    print("Set dry_run=False to actually delete folders.")
else:
    print("No workers loaded - nothing to clean up.")

CLEANUP PREVIEW (dry_run=True):
Share: \\192.168.3.8\RasRemote\
  Folders: 68
  [WOULD DELETE] BaldEagleDamBrk_10_SW1_0d970906 (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_10_SW1_f8ae583e (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_11_SW2_2cf5ce15 (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_11_SW2_d5d823d6 (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_12_SW3_d4137c14 (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_14_SW4_3996e252 (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_16_SW1_9b4d5e9a (338.8 MB)
  [WOULD DELETE] BaldEagleDamBrk_16_SW1_d575be09 (337.9 MB)
  [WOULD DELETE] BaldEagleDamBrk_20_SW2_ab6c8c81 (337.9 MB)
  [WOULD DELETE] BaldEagleDamBrk_20_SW2_ba9a2113 (338.8 MB)
  [WOULD DELETE] BaldEagleDamBrk_21_SW3_21f0530e (338.8 MB)
  [WOULD DELETE] BaldEagleDamBrk_21_SW3_add187ed (337.9 MB)
  [WOULD DELETE] BaldEagleDamBrk_26_SW1_99b240c3 (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_26_SW1_a7f83eac (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_27_SW2_026418cc (0.0 MB)
  [WOULD DELETE] BaldEagleDamBrk_28_SW3_

---
# Summary: Local vs Remote Execution

| Feature | Local (`compute_parallel`) | Remote (`compute_parallel_remote`) |
|---------|---------------------------|------------------------------------|
| Setup | None | Configure RemoteWorkers.json |
| Execution | Local machine only | Remote machines via PsExec/Docker |
| File Transfer | Direct | Via network shares |
| Scaling | Limited by local cores | Unlimited remote machines |
| Best For | Small jobs, testing | Large batches, distributed work |

## Quick Reference

**Local Parallel:**
```python
results = RasCmdr.compute_parallel(
    plan_number=["01", "02", "03"],
    max_workers=3,
    num_cores=2,
    ras_object=ras
)
```

**Remote Parallel:**
```python
workers = load_workers_from_json("RemoteWorkers.json")
results = compute_parallel_remote(
    plan_numbers=["01", "02", "03"],
    workers=workers,
    num_cores=4
)
```

---

**For complete setup instructions, see:**
- `feature_dev_notes/RasRemote/REMOTE_WORKER_SETUP_GUIDE.md`